网站常见的反爬虫和应对方法
一般网站从三个方面反爬虫：用户请求的Headers，用户行为，网站目录和数据加载方式。
前两种比较容易遇到，大多数网站都从这些角度来反爬虫。第三种一些应用ajax的网站会采用，这样增大了爬取的难度。


① 通过Headers反爬虫
从用户请求的Headers反爬虫是最常见的反爬虫策略。
很多网站都会对Headers的User-Agent进行检测，还有一部分网站会对Referer进行检测（一些资源网站的防盗链就是检测Referer）。

对于检测Headers的反爬虫，在爬虫中修改或者添加Headers就能很好的绕过。

将浏览器的User-Agent复制到爬虫的Headers中；或者将Referer值修改为目标网站域名。
② 基于用户行为反爬虫
还有一部分网站是通过检测用户行为，例如同一IP短时间内多次访问同一页面，或者同一账户短时间内多次进行相同操作。
大多数网站都是前一种情况，对于这种情况，使用IP代理就可以解决。
对于第二种情况，可以在每次请求后随机间隔几秒再进行下一次请求。
有些有逻辑漏洞的网站，可以通过请求几次，退出登录，重新登录，继续请求来绕过同一账号短时间内不能多次进行相同请求的限制。
③ 动态页面的反爬虫
大多网站界面都是静态页面(即在浏览器中查看源代码可见)，但是还有一部分网站中的数据是后期通过ajax请求(或其他方式如推送技术)得到。
解决办法：首先用Firebug或者Fiddler对网络请求进行分析。找到ajax的请求url，通过Python模拟请求得到需要的数据。

但是还有些网站把ajax请求的所有参数全部加密了，针对于这方式我们后面会给大家讲解动态渲染页面信息爬取。



1)请求(客户端->服务端[request])
    GET(请求的方式) /newcoder/hello.html(请求的目标资源) HTTP/1.1(请求采用的协议和版本号)
    Accept: */*(客户端能接收的资源类型)
    Accept-Language: en-us(客户端接收的语言类型)
    Connection: Keep-Alive(维护客户端和服务端的连接关系)
    Host: localhost:8080(连接的目标主机和端口号)
    Referer: http://localhost/links.asp(告诉服务器我来自于哪里)
    User-Agent: Mozilla/4.0(客户端版本号的名字)
    Accept-Encoding: gzip, deflate(客户端能接收的压缩数据的类型)
    If-Modified-Since: Tue, 11 Jul 2000 18:23:51 GMT(缓存时间)
    Cookie(客户端暂存服务端的信息)
    Date: Tue, 11 Jul 2000 18:23:51 GMT(客户端请求服务端的时间)

2)响应(服务端->客户端[response])
    HTTP/1.1(响应采用的协议和版本号) 200(状态码) OK(描述信息)
    Location: http://www.baidu.com(服务端需要客户端访问的页面路径)
    Server:apache tomcat(服务端的Web服务端名)
    Content-Encoding: gzip(服务端能够发送压缩编码类型)
    Content-Length: 80(服务端发送的压缩数据的长度)
    Content-Language: zh-cn(服务端发送的语言类型)
    Content-Type: text/html; charset=GB2312(服务端发送的类型及采用的编码方式)
    Last-Modified: Tue, 11 Jul 2000 18:23:51 GMT(服务端对该资源最后修改的时间)
    Refresh: 1;url=http://www.it315.org(服务端要求客户端1秒钟后，刷新，然后访问指定的页面路径)
    Content-Disposition: attachment; filename=aaa.zip(服务端要求客户端以下载文件的方式打开该文件)
    Transfer-Encoding: chunked(分块传递数据到客户端）
    Set-Cookie:SS=Q0=5Lb_nQ; path=/search(服务端发送到客户端的暂存数据)
    Expires: -1//3种(服务端禁止客户端缓存页面数据)
    Cache-Control: no-cache(服务端禁止客户端缓存页面数据)
    Pragma: no-cache(服务端禁止客户端缓存页面数据)
    Connection: close(1.0)/(1.1)Keep-Alive(维护客户端和服务端的连接关系)

    Date: Tue, 11 Jul 2000 18:23:51 GMT(服务端响应客户端的时间)

In [4]:
import requests

# 代理IP地址
proxy = {'HTTP':'117.85.105.170:808','HTTPS':'117.85.105.170:808'}
# header头信息


#user_agents
user_agents = [
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    'Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7',
    'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0',
]
import random
agent = random.choice(user_agents) # 随机获取一个浏览器用户信息


headers = {
    'Host': 'blog.csdn.net',
    'User-Agent': agent,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'zh-CN,zh;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate',
    'Referer': 'http://www.baidu.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
}
# 请求url地址
url = "http://blog.csdn.net"



# 提交请求爬取信息
response = requests.get(url,headers=headers,proxies=proxy)

# 获取响应码
print(response.status_code)

200


In [2]:
#最好的网络爬虫的解析框架
#https://getiot.tech/manual/PythonNotes/05_spider_base/week02/1.2.html


# 创建解析器
from bs4 import BeautifulSoup



soup = BeautifulSoup(content,"lxml")

# 输出网页内容：注：此内容已被缩进格式化（自动更正格式），其实这个是在上一步实例化时就已完成
print(soup.prettify())

#输出网页中title标签中的内容
print(soup.title.string)

<!doctype html><html lang="zh" data-server-rendered="true"><head><title>CSDN博客-专业IT技术发表平台</title> <meta name="keywords" content="CSDN"> <meta name="description" content="CSDN是全球知名中文IT技术交流平台,创建于1999年,包含原创博客、精品问答、职业培训、技术论坛、资源下载等产品服务,提供原创、优质、完整内容的专业IT技术开发社区."> <meta http-equiv="content-type" content="text/html;charset=utf-8"> <meta name="viewport" content="initial-scale=1, maximum-scale=1, user-scalable=no, minimal-ui"> <meta name="referrer" content="always"> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <link rel="shortcut icon" href="https://g.csdnimg.cn/static/logo/favicon32.ico" type="image/x-icon"> <link rel="canonical" href="http://blog.csdn.net"> <!----> 
          <meta name="toolbar" content={"type":"0","fixModel":"1"} />
       
          <meta name="report" content={"spm":"1001.2100"} />
       <script src="https://g.csdnimg.cn/??lib/jquery/1.12.4/jquery.min.js,user-tooltip/2.7/user-tooltip.js,lib/qrcode/1.0.0/qrcode.min.js"></script> <script src='//g.csdnimg.cn/commo

Scrapy: Scrapy是一个强大的开源Web爬虫框架，它提供了高度的可定制性和灵活性。它支持异步网络请求、自动处理cookies和session等功能。Scrapy也提供了方便的数据提取功能，并且可以很好地处理大规模爬取任务。

Beautiful Soup: Beautiful Soup是一个优秀的HTML和XML解析库，它允许您从网页中提取数据，并提供了简单易用的API。Beautiful Soup通常与请求库（如requests）结合使用，用于解析和提取网页内容。

Selenium: Selenium是一个用于自动化浏览器操作的工具，通常用于处理动态网页和JavaScript渲染的页面。它模拟用户在浏览器中的操作，可以执行点击、填写表单等操作，并获取渲染后的内容。

Requests-HTML: Requests-HTML是一个基于requests库的HTML解析工具，使您能够使用类似于jQuery的语法来解析网页内容。它结合了请求和解析功能，是处理简单网页爬取任务的不错选择。

PyQuery: PyQuery是一个类似于jQuery的解析库，它使得在Python中使用CSS选择器来提取和操作HTML文档变得更加简单。

Splash: Splash是一个JavaScript渲染服务，可用于处理JavaScript渲染的网页。它与Scrapy和其他爬虫框架集成良好，可以解决一些动态页面爬取的问题。

Apify: Apify是一个用于构建Web爬虫和自动化任务的云平台，它提供了简单易用的API和工具，使得爬虫的开发和管理更加便捷。

Goutte: Goutte是一个用于PHP的简单Web爬虫库，如果您使用PHP语言进行爬虫开发，它是一个不错的选择。